<a href="https://colab.research.google.com/github/MaroonBlue/Speech_Commands_with_RNNs/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment setup

## Cloning repository

In [ ]:
%cd /content/
!rm -r ./Speech_Commands_with_RNNs/
!git clone https://github.com/MaroonBlue/Speech_Commands_with_RNNs.git
%cd Speech_Commands_with_RNNs/

/content
Cloning into 'Speech_Commands_with_RNNs'...
remote: Enumerating objects: 273, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 273 (delta 35), reused 46 (delta 24), pack-reused 213
Receiving objects: 100% (273/273), 1.38 MiB | 1.83 MiB/s, done.
Resolving deltas: 100% (156/156), done.
/content/Speech_Commands_with_RNNs


## Installing packages

In [ ]:
!pip install -r requirements.txt
!pip install -U -q tensorflow tensorflow_datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Downloading dataset

## Download form URL

In [ ]:
!python utils/download_dataset.py

2023-04-23 15:14:06.285207: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-23 15:14:07.225078: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
1121103842/1121103842 [==============================] - 67s 0us/step
Moved 6798 files to the validation directory
Moved 6835 files to the test directory


## Generate and save training and validation data

In [ ]:
!python ./utils/dataset.py

2023-04-23 15:16:59.025208: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-23 15:16:59.980799: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Training dataset: 100% 22536/22536 [04:36<00:00, 81.63it/s]
Validation dataset: 100% 6798/6798 [01:03<00:00, 106.58it/s]


## Copy .npy files to drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
!cp -R saved_data ../drive/MyDrive/project_data

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Copy from drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/project_data')
!cp -R ../content/drive/project_data saved_data

## Copy from local

In [ ]:
!cp -R data ../data

In [ ]:
!cp -R ../data data

In [ ]:
!cp ./saved_data/X_t.npy ../X_t.npy
!cp ./saved_data/y_t.npy ../y_t.npy
!cp ./saved_data/X_v.npy ../X_v.npy
!cp ./saved_data/y_v.npy ../y_v.npy

In [ ]:
!cp ../X_t.npy ./saved_data/X_t.npy
!cp ../y_t.npy ./saved_data/y_t.npy
!cp ../X_v.npy ./saved_data/X_v.npy
!cp ../y_v.npy ./saved_data/y_v.npy

# Experiments

## Setup and imports

In [ ]:
%cd Speech_Commands_with_RNNs/

[Errno 2] No such file or directory: 'Speech_Commands_with_RNNs/'
/content/Speech_Commands_with_RNNs


In [ ]:
import os
import wandb
import tensorflow as tf
import sys
import numpy as np

from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from numpy import load

sys.path.append("./")
from utils.utils import set_seeds, make_configs, step_decay
from models.test_model import get_test_model
from models.CNN_LSTM import CNN_LSTM1
from models.GRU import GRU1, GRU2, GRU3
from models.LSTM import LSTM1, LSTM2, LSTM3
from models.simpleRNN import simpleRNN1, simpleRNN2, simpleRNN3

import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

In [ ]:
!wandb offline

W&B offline. Running your script from this directory will only write metadata locally. Use wandb disabled to completely turn off W&B.


## Configs and data

In [ ]:
ENTITY = 'wo-team'
PROJECT = 'dl-rnn-audio'
GROUP = 'simple-lstm-gru'
NAME = '_'
SAVE_PATH = 'weights/'

models = {
    'SimpleRNN1': simpleRNN1,
    'SimpleRNN2': simpleRNN2,
    'SimpleRNN3': simpleRNN3,
    'LSTM1': LSTM1,
    'LSTM2': LSTM2,
    'LSTM3': LSTM3,
    'GRU1': GRU1,
    'GRU1': GRU2,
    'GRU1': GRU3,
}

base_config = {
    'dataloader': {
        'sample_rate': 8000,
        'unknown_silence_samples': 2000,
        'seed': 0,
        'batch_size': 128,
        'convert_to_image': False,
    },
    'training': {
        'n_epochs': 50,
        'dropout': 0.3,
    },
    'compile':{
        'loss': 'sparse_categorical_crossentropy',
        'optimizer': 'adam',
        'metrics': ['accuracy', 'sparse_categorical_accuracy']
    },
    'model': {
        'architecture': 'SimpleRNN1',
        'model_init': None,
        'id': None,
        'save_path': None,
    },
    'early_stopper':{
        'monitor': 'val_sparse_categorical_accuracy',
        'min_delta': 0.001,
        'patience': 4,
        'verbose': 1,
        'start_from_epoch': 10,
        'restore_best_weights': True,
    },
    'checkpointer':{
        'monitor': 'val_sparse_categorical_accuracy',
        "verbose": 1,
        'save_best_only': True
    },
    'scheduler': LearningRateScheduler(step_decay),
    'other':{
            'num_classes':12,
    }
}

combinations = {
    'seeds': {
        'dict_path': ['dataloader', 'seed'],
        'values': [0, 1, 2]
    },
    'architecture':{
        'dict_path': ['model', 'architecture'],
        'values': ['SimpleRNN1','SimpleRNN2','SimpleRNN3', 'LSTM1', 'LSTM2', 'LSTM3', 'GRU1', 'GRU2', 'GRU3'],
    },
    'dropout': {
        'dict_path': ['training', 'dropout'],
        'values': [0.3, 0.5]
    }

}



configs = make_configs(base_config, combinations)

LOAD_PTH = "./saved_data/"
X_t = load(LOAD_PTH + "X_t.npy")
y_t = load(LOAD_PTH + "y_t.npy")
X_v = load(LOAD_PTH + "X_v.npy")
y_v = load(LOAD_PTH + "y_v.npy")
y_t = np.argmax(y_t, axis=1).transpose()
y_v = np.argmax(y_v, axis=1).transpose()

## Training

In [ ]:
start_config = int(input("Provide ID of first config: "))

for i, config in enumerate(configs):
    if i < start_config:
        continue

    set_seeds(config['dataloader']['seed'])
    config['model']['id'] = i
    NAME = str(config['model']['id']) + config['model']['architecture'] + "_seed" + str(config['dataloader']['seed']) +"_dropout"+str(config['training']['dropout']).replace(".", ",")
    config['model']['model_init'] = models[config['model']['architecture']]

    wandb.init(
        project = PROJECT,
        entity = ENTITY,
        group = GROUP,
        name = NAME,
        config = config)
    
    l = len(configs)
    print(f"---------------\nConfig {i+1}/{l}\n---------------\n\n")
    print('Running config:', config, "\n")

    input_shape = X_t.shape[1:]

    model = config['model']['model_init'](input_shape=input_shape, dropout=config['training']['dropout'])

    model.compile(**config["compile"])
    earlystopper = EarlyStopping(**config["early_stopper"])
    checkpointer = ModelCheckpoint(NAME+'.h5', **config["checkpointer"])
    lrate = config["scheduler"]

    history = model.fit(
                X_t,
                y_t,
                epochs=config['training']['n_epochs'],
                validation_data=(X_v, y_v),
                batch_size=config['dataloader']['batch_size'],
                shuffle=True,
                callbacks=[
                    earlystopper, 
                    checkpointer, 
                    lrate,
                    WandbMetricsLogger(log_freq=5),
                    WandbModelCheckpoint("weights/wandb")
                ])
    save_path = os.path.join(SAVE_PATH, NAME)
    model.save(save_path)

    wandb.finish()

Provide ID of first config: 0


batch/accuracy,▁▁▂▃▄▅▆▆▆▇▇▇██
batch/batch_step,▁▂▂▃▃▄▄▅▅▆▆▇▇█
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,██▇▆▅▄▄▃▃▂▂▂▁▁
batch/sparse_categorical_accuracy,▁▁▂▃▄▅▆▆▆▇▇▇██
batch/accuracy,0.31143
batch/batch_step,65
batch/learning_rate,0.001
batch/loss,1.95446
batch/sparse_categorical_accuracy,0.31143


---------------
Config 1/54
---------------


Running config: {'dataloader': {'sample_rate': 8000, 'unknown_silence_samples': 2000, 'seed': 0, 'batch_size': 128, 'convert_to_image': False}, 'training': {'n_epochs': 50, 'dropout': 0.3}, 'compile': {'loss': 'sparse_categorical_crossentropy', 'optimizer': 'adam', 'metrics': ['accuracy', 'sparse_categorical_accuracy']}, 'model': {'architecture': 'SimpleRNN1', 'model_init': <function simpleRNN1 at 0x7f30039020d0>, 'id': 0, 'save_path': None}, 'early_stopper': {'monitor': 'val_sparse_categorical_accuracy', 'min_delta': 0.001, 'patience': 4, 'verbose': 1, 'start_from_epoch': 10, 'restore_best_weights': True}, 'checkpointer': {'monitor': 'val_sparse_categorical_accuracy', 'verbose': 1, 'save_best_only': True}, 'scheduler': <keras.callbacks.LearningRateScheduler object at 0x7f2ee6d2ea30>, 'other': {'num_classes': 12}} 

Changing learning rate to 0.001
Epoch 1/50
160/160 [==============================] - ETA: 0s - loss: 2.3819 - accuracy: 0.195

wandb: Adding directory to artifact (./weights/wandb)... Done. 0.0s


160/160 [==============================] - 69s 404ms/step - loss: 2.3819 - accuracy: 0.1951 - sparse_categorical_accuracy: 0.1951 - val_loss: 2.4513 - val_accuracy: 0.0508 - val_sparse_categorical_accuracy: 0.0508 - lr: 0.0010
Changing learning rate to 0.001
Epoch 2/50
 18/160 [==>...........................] - ETA: 40s - loss: 2.3484 - accuracy: 0.2018 - sparse_categorical_accuracy: 0.2018

KeyboardInterrupt: ignored